In [1]:
print(type(int))

<class 'type'>


In [2]:
int_value = 10

In [3]:
MMBtu = int(float(input("How many MMBtu of natural gas do you want to buy?: ")))


How many MMBtu of natural gas do you want to buy?:  1


In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load the dataset
file_path = "Nat_Gas.xlsx"  # Use CSV if necessary: "Nat_Gas.csv"
df = pd.read_excel(file_path, parse_dates=['Dates'])

# Extract dates and prices from the DataFrame
dates = df['Dates'].tolist()
prices = df['Prices'].astype(float).tolist()

# Function to find price using interpolation
def natGasPrice(find, day):
    lower = allprices[find]
    upper = allprices[find + 1]
    return np.interp(day, [0, 30.5], [lower, upper])  # 30.5 avg days per month

# Function to convert date to index
def findDate(month, day, year):
    find = 0
    for i in range(year - 2021):
        find += 12
    find += month
    return find if year != 2020 else (0 if month == 10 else (1 if month == 11 else 2))

# Ask for gas volume input
MMBtu = int(float(input("How many MMBtu of natural gas do you want to buy?: ")))

# Get injection date
while True:
    print("\nEnter an injection date between 10/31/2020 and 10/31/2025:")
    yearinj = int(input("Enter year: "))
    monthinj = int(input("Enter month: "))
    dayinj = int(input("Enter day: "))
    
    if (2020 <= yearinj <= 2025) and (1 <= monthinj <= 12) and (1 <= dayinj <= 31):
        break
    else:
        print("\nInvalid date. Try again.")

# Get withdrawal date
while True:
    print(f"\nEnter a withdrawal date after {monthinj}/{dayinj}/{yearinj} up to 10/31/2025:")
    yearwith = int(input("Enter year: "))
    monthwith = int(input("Enter month: "))
    daywith = int(input("Enter day: "))
    
    if (2020 <= yearwith <= 2025) and (1 <= monthwith <= 12) and (1 <= daywith <= 31) and findDate(monthwith, daywith, yearwith) > findDate(monthinj, dayinj, yearinj):
        break
    else:
        print("\nInvalid date. Must be after injection date.")

# Convert data into arrays
num_years = len(prices) // 12  # Total years in dataset
larray = np.array(prices[: num_years * 12]).reshape(num_years, 12)  # Reshape prices into years × months

# Compute monthly price changes
avgmonth = np.zeros((12, 3))
for i in range(12):
    if num_years >= 4:
        avgmonth[i] = np.diff(larray[-4:, i])  # Extract last 4 years of monthly prices
    else:
        avgmonth[i][: num_years - 1] = np.diff(larray[:, i])  # Use available years

# Compute extrapolated prices
extprices = prices[-12:]
for i in range(12):
    extprices[i] += np.mean(avgmonth[i][: num_years - 1])  # Use only valid years

# Calculate total costs and profits
MMBtuPrice = MMBtu * 1000  # Convert to price per thousand
allprices = np.concatenate((prices, extprices))  # Append extrapolated prices

# Get date indexes
foundDateInj = findDate(monthinj, dayinj, yearinj)
foundDateWith = findDate(monthwith, daywith, yearwith)

# Compute prices
injPrice = round(natGasPrice(foundDateInj, dayinj), 2)
withPrice = round(natGasPrice(foundDateWith, daywith), 2)

# Compute total costs
injPriceTot = round(injPrice * MMBtuPrice, 3)
withPriceTot = round(withPrice * MMBtuPrice, 3)
extractionPrice = MMBtu * 10 * 2  # Injection & withdrawal
storageTime = foundDateWith - foundDateInj + 1
storageTimePrice = storageTime * 10  # Cost per month

# Display results
print(f"\nPurchase price on {monthinj}/{dayinj}/{yearinj}: ${injPrice}")
print(f"Selling price on {monthwith}/{daywith}/{yearwith}: ${withPrice}")
print(f"Total Purchase Cost: ${injPriceTot}k")
print(f"Total Selling Revenue: ${withPriceTot}k")
print(f"Injection & Withdrawal Cost: ${extractionPrice}k")
print(f"Storage Cost for {storageTime} months: ${storageTimePrice}k")
print("\nTransportation cost: $100k")

# Compute total profit
totalProfit = round(withPriceTot - injPriceTot - storageTimePrice - 100 - extractionPrice, 3)
print(f"\nTotal Profit: ${totalProfit}k")


How many MMBtu of natural gas do you want to buy?:  1



Enter an injection date between 10/31/2020 and 10/31/2025:


Enter year:  2021
Enter month:  2
Enter day:  1



Enter a withdrawal date after 2/1/2021 up to 10/31/2025:


Enter year:  2023
Enter month:  1
Enter day:  1



Purchase price on 2/1/2021: $11.0
Selling price on 1/1/2023: $11.6
Total Purchase Cost: $11000.0k
Total Selling Revenue: $11600.0k
Injection & Withdrawal Cost: $20k
Storage Cost for 24 months: $240k

Transportation cost: $100k

Total Profit: $240.0k
